#eager执行基础

这是使用TensorFlow的入门教程。它将涵盖：

- 导入所需的包
- 创建和使用张量
- 使用GPU加速
- 数据集

##导入TensorFlow
首先，导入tensorflow模块并启用急切执行。渴望执行可以为TensorFlow提供更具交互性的前端，我们将在稍后讨论其详细信息。

In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

tf.enable_eager_execution()

## 张量
张量是一个多维数组。与NumPy ndarray对象类似，Tensor对象具有数据类型和形状。此外，Tensors可以驻留在加速器（如GPU）内存中。TensorFlow提供了丰富的操作库（`tf.add，tf.matmul，tf.linalg.inv`等），它们使用和生成Tensors。这些操作自动转换本机Python类型。例如：

In [2]:
print(tf.add(1,2))
print(tf.add([1,2],[3,4]))
print(tf.square(5))
print(tf.reduce_sum([1,2,3]))
print(tf.encode_base64("hello world"))

#Operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)
tf.Tensor(13, shape=(), dtype=int32)


每个Tensor都有一个形状和一个数据类型

In [3]:
x = tf.matmul([[1]],[[2,3]])
print(x.shape)
print(x.dtype)

(1, 2)
<dtype: 'int32'>


In [4]:
x

<tf.Tensor: id=30, shape=(1, 2), dtype=int32, numpy=array([[2, 3]], dtype=int32)>

NumPy阵列和TensorFlow张量之间最明显的区别是：

1. 张量可以由加速器内存（如GPU，TPU）支持。
2. 张量是不可改变的。

##NumPy兼容性

TensorFlow张量和NumPy ndarrays之间的转换非常简单，如：

- TensorFlow操作自动将NumPy ndarrays转换为Tensors。
- NumPy操作自动将Tensors转换为NumPy ndarrays。

通过调用.numpy()它们的方法，可以将张量显式转换为NumPy ndarrays 。这些转换通常很便宜，因为如果可能，数组和Tensor共享底层内存表示。但是，共享底层表示并不总是可行的，因为Tensor可以托管在GPU内存中，而NumPy阵列总是由主机内存支持，因此转换将涉及从GPU到主机内存的复制。

In [5]:
import numpy as np

ndarray = np.ones([3,3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)

print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


##GPU加速
通过使用GPU进行计算，可以加速许多TensorFlow操作。在没有任何注释的情况下，TensorFlow会自动决定是使用GPU还是CPU进行操作（如果需要，还可以复制CPU和GPU内存之间的张量）。由操作产生的张量通常由执行操作的设备的存储器支持。例如：

In [6]:
x = tf.random_uniform([3,3])

print("Is there a GPU available: "),
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0: "),
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
True
Is the Tensor on GPU #0: 
True


##设备名称
该Tensor.device属性提供托管张量内容的设备的完全限定字符串名称。此名称编码许多详细信息，例如正在执行此程序的主机的网络地址的标识符以及该主机中的设备。这是分布式执行TensorFlow程序所必需的。GPU:<N>如果张量位于N主机上的第-GPU上，则字符串结束。

##显式设备放置
TensorFlow中的术语“放置”指的是如何为执行设备分配（放置）各个操作。如上所述，当没有提供明确的指导时，TensorFlow会自动决定执行操作的设备，并在需要时将Tensors复制到该设备。但是，可以使用tf.device上下文管理器将TensorFlow操作显式放置在特定设备上。例如：

In [7]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x,x)
    
  result = time.time() - start
  
  print("10 loops:  {:0.2f}ms".format(1000 * result))
  
  
#Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random_uniform([1000,1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)
  
  
#Force execution on GPU #0 if available
if tf.test.is_gpu_available():
  with tf.device("GPU:0"):# Or GPU:1 for the 2nd GPU,GPU:2 for the 3rd etc.
    x = tf.random_uniform([1000,1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

On CPU:
10 loops:  227.69ms
10 loops:  684.68ms


## 数据集
本节演示了使用 tf.data.Dataset用于构建管道以将数据提供给模型的API。它涵盖：

- 创建一个Dataset。
- 在Dataset启用了急切执行的情况下进行迭代。

我们建议使用Datasets API从简单，可重复使用的部分构建高性能，复杂的输入管道，这些部分将为模型的培训或评估循环提供支持。

如果您熟悉TensorFlow图，则在Dataset启用eager执行时，构建对象的API 保持完全相同，但迭代数据集元素的过程稍微简单一些。您可以对tf.data.Dataset对象使用Python迭代，而不需要显式创建tf.data.Iterator对象。因此，在启用eager执行时，TensorFlow Guide中对迭代器的讨论无关紧要。

#### 创建一个源 Dataset
使用其中一个工厂函数创建源数据集Dataset.from_tensors， Dataset.from_tensor_slices或者使用从TextLineDataset或等文件中读取的对象TFRecordDataset。有关详细信息，请参阅TensorFlow指南。

In [0]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6])

#Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
  Line 2
  Line 3
  """)
  
  ds_file = tf.data.TextLineDataset(filename)

## 应用转换
使用如转换功能map，batch，shuffle等来转换应用到数据集的记录。有关详细信息，请参阅API文档tf.data.Dataset。

In [0]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

##重复
当启用eager执行时，Dataset对象支持迭代。如果您熟悉Dataset在TensorFlow图中使用s，请注意不需要呼叫Dataset.make_one_shot_iterator()或get_next()呼叫。

In [10]:
print("Elements of ds_tensors:")
for x in ds_tensors:
  print(x)
  
print("\nElements in ds_file:")
for x in ds_file:
  print(x)

Elements of ds_tensors:
tf.Tensor([4 1], shape=(2,), dtype=int32)
tf.Tensor([ 9 16], shape=(2,), dtype=int32)
tf.Tensor([36 25], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'  Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'  Line 3' b'  '], shape=(2,), dtype=string)
